## Define LLM and Embedding model

In [ ]:
# !pip install llama-index

In [ ]:
# CMAKE_ARGS="-DLLAMA_CUDA=on"

In [ ]:
# pip install llama-cpp-python

In [ ]:
# %pip install llama-index-embeddings-huggingface
# %pip install llama-index-llms-llama-cpp

In [ ]:
# !pip install  tensorboard
# # Install Hugging Face libraries
# !pip install --upgrade \
#   "sentence-transformers>=3" \
#   "datasets==2.19.1"  \
#   "transformers==4.41.2" 

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

embed_model = HuggingFaceEmbedding(
    model_name="maiduchuy321/vietnamese-bi-encoder-fine-tuning-for-law-chatbot")
# embed_model = HuggingFaceEmbedding(
#     model_name="bkai-foundation-models/vietnamese-bi-encoder"
# )


C:\Users\maidu\anaconda3\envs\LamaIndex\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
Settings.embed_model = embed_model

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

In [ ]:
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url="https://huggingface.co/uonlp/Vistral-7B-Chat-gguf/resolve/main/ggml-vistral-7B-chat-q4_0.gguf?download=true"
    ,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    # model_path="model_path",
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=2048,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 30},
    # transform inputs into Llama2 format
    # messages_to_prompt=messages_to_prompt,
    # completion_to_prompt=completion_to_prompt,
    verbose=True,
)

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from I:/Hoc/do an/model/ggml-vistral-7B-chat-q4_1.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_

In [ ]:
Settings.llm = llm

In [ ]:
# response = llm.complete("Năm nay là năm bao nhiêu")
# print(response.text)

## Ingestion

### Load data

In [ ]:
import pandas as pd

# Đường dẫn đến file Excel
file_path = 'CauHoi_DVC.HaGiang_ThucTeTaiTinh.xlsx'

# Đọc tất cả các sheet vào một dictionary
excel_data = pd.read_excel(file_path, sheet_name=None)

# Tạo các DataFrame riêng biệt cho mỗi sheet
df_sheet1 = excel_data.get('Sở, ngành')
df_sheet2 = excel_data.get('UBND các huyện, tp')
df_sheet3 = excel_data.get('Các đơn vị đã gửi câu trả lời')

In [ ]:
df_sheet1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 869 entries, 0 to 868
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   STT              864 non-null    float64
 1   Tên cơ quan      864 non-null    object 
 2   Tên đơn vị       77 non-null     object 
 3   Câu hỏi          864 non-null    object 
 4   Câu trả lời      864 non-null    object 
 5   Tên TTHC         745 non-null    object 
 6   Số hiệu văn bản  864 non-null    object 
 7   Ghi Chú          0 non-null      float64
dtypes: float64(2), object(6)
memory usage: 54.4+ KB


In [ ]:
df_sheet2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 763 entries, 0 to 762
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   STT              763 non-null    int64  
 1   Tên cơ quan      763 non-null    object 
 2   Tên đơn vị       749 non-null    object 
 3   Câu hỏi          763 non-null    object 
 4   Câu trả lời      763 non-null    object 
 5   Tên TTHC         638 non-null    object 
 6   Số hiệu văn bản  763 non-null    object 
 7   Ghi Chú          0 non-null      float64
dtypes: float64(1), int64(1), object(6)
memory usage: 47.8+ KB


In [ ]:
df_sheet1.drop(df_sheet1.index[-5:], inplace=True)

In [ ]:
df_sheet1

,STT,Tên cơ quan,Tên đơn vị,Câu hỏi,Câu trả lời,Tên TTHC,Số hiệu văn bản,Ghi Chú
0,1.0,Ban quản lý khu kinh tế,NaN,Đối tượng nào được thực hiện Thủ tục: Giao đất...,"Tổ chức, cơ sở tôn giáo, người Việt Nam định c...",Mã thủ tục: 1.002253,17/BQLKKT-VP ngày 09/01/2024,NaN
1,2.0,Ban quản lý khu kinh tế,NaN,"Phí lệ phí để thực hiện Thủ tục: Giao đất, cho...","Thủ tục HC không có phí, lệ phí (phí, lệ phí =...",Mã thủ tục: 1.002253,17/BQLKKT-VP ngày 09/01/2024,NaN
2,3.0,Ban quản lý khu kinh tế,NaN,"Thời gian thực hiện Thủ tục: Giao đất, cho th...",Không quá 20 ngày làm việc kể từ ngày nhận đủ ...,"Mã thủ tục: 1.002253\nTên thủ tục: Giao đất, c...",17/BQLKKT-VP ngày 09/01/2024,NaN
3,4.0,Ban quản lý khu kinh tế,NaN,Thời gian thực hiện Thủ tục: Thẩm định thiết ...,Theo quy định của pháp luật:Không quá 40 ngày ...,Mã thủ tục: 1.009973\nTên thủ tục: Thẩm định t...,17/BQLKKT-VP ngày 09/01/2024,NaN
4,5.0,Ban quản lý khu kinh tế,NaN,"Văn bản nào quy định với phí, lệ phí Thẩm định...",Theo quy định tại Thông tư số 210/2016/TT-BTC ...,Mã thủ tục: 1.009973\nTên thủ tục: Thẩm định t...,17/BQLKKT-VP ngày 09/01/2024,NaN
...,...,...,...,...,...,...,...,...
859,860.0,Sở Y tế,NaN,"Bác sỹ, y sỹ, điều dưỡng, kỹ thuật viên y tế l...","Thông tin về thẩm quyền cấp mới, cấp lại, gia ...","Cấp mới, cấp lại, gia hạn, điều chỉnh giấy phé...",176/SYT-VP ngày 24/01/2024,NaN
860,861.0,Sở Y tế,NaN,"Thời gian khám bệnh, chữa bệnh để làm người ch...","Theo Điều 104 của Luật Khám bệnh, chữa bệnh, c...",Điều chỉnh giấy phép hoạt động chuyên môn khi ...,176/SYT-VP ngày 24/01/2024,NaN
861,862.0,Sở Y tế,NaN,Trình tự thực hiện tự công bố cơ sở đủ điều ki...,Trình tự thực hiện tự công bố cơ sở đủ điều ki...,Trình tự thực hiện tự công bố cơ sở đủ điều ki...,176/SYT-VP ngày 24/01/2024,NaN
862,863.0,Sở Y tế,NaN,Cơ sở y tế muốn được đánh giá năng lực thực hi...,Theo quy định tại khoản 4 Điều 17 Nghị định số...,NaN,176/SYT-VP ngày 24/01/2024,NaN


In [ ]:
df = pd.concat([df_sheet1, df_sheet2], ignore_index=True)
df

,STT,Tên cơ quan,Tên đơn vị,Câu hỏi,Câu trả lời,Tên TTHC,Số hiệu văn bản,Ghi Chú
0,1.0,Ban quản lý khu kinh tế,NaN,Đối tượng nào được thực hiện Thủ tục: Giao đất...,"Tổ chức, cơ sở tôn giáo, người Việt Nam định c...",Mã thủ tục: 1.002253,17/BQLKKT-VP ngày 09/01/2024,NaN
1,2.0,Ban quản lý khu kinh tế,NaN,"Phí lệ phí để thực hiện Thủ tục: Giao đất, cho...","Thủ tục HC không có phí, lệ phí (phí, lệ phí =...",Mã thủ tục: 1.002253,17/BQLKKT-VP ngày 09/01/2024,NaN
2,3.0,Ban quản lý khu kinh tế,NaN,"Thời gian thực hiện Thủ tục: Giao đất, cho th...",Không quá 20 ngày làm việc kể từ ngày nhận đủ ...,"Mã thủ tục: 1.002253\nTên thủ tục: Giao đất, c...",17/BQLKKT-VP ngày 09/01/2024,NaN
3,4.0,Ban quản lý khu kinh tế,NaN,Thời gian thực hiện Thủ tục: Thẩm định thiết ...,Theo quy định của pháp luật:Không quá 40 ngày ...,Mã thủ tục: 1.009973\nTên thủ tục: Thẩm định t...,17/BQLKKT-VP ngày 09/01/2024,NaN
4,5.0,Ban quản lý khu kinh tế,NaN,"Văn bản nào quy định với phí, lệ phí Thẩm định...",Theo quy định tại Thông tư số 210/2016/TT-BTC ...,Mã thủ tục: 1.009973\nTên thủ tục: Thẩm định t...,17/BQLKKT-VP ngày 09/01/2024,NaN
...,...,...,...,...,...,...,...,...
1622,759.0,UBND thành phố Hà Giang,UBND thành phố,"Trình tự, thủ tục điều chỉnh giấy phép xây dự...","1.\tTrình tự, thủ tục thực hiện:\n1.1\tGửi (nộ...",Thủ tục xin cấp giấy phép xây dựng\nnhà ở riê...,3370/UBND-VP ngày 30/12/2024,NaN
1623,760.0,UBND thành phố Hà Giang,UBND thành phố,"Trình tự, thủ tục cấp lại giấy phép xây dựng ...","1.\tTrình tự, thủ tục thực hiện:\n1.1\tGửi (nộ...",Thủ tục xin cấp giấy phép xây dựng nhà ở riên...,3370/UBND-VP ngày 30/12/2024,NaN
1624,761.0,UBND thành phố Hà Giang,UBND thành phố,"Trình tự, thủ tục xin cấp giấy phép xây dựng ...","1. Trình tự, thủ tục thực hiện:\n1.1 Gửi (nộp)...",Thủ tục xin cấp giấy phép xây dựng nhà ở riên...,3370/UBND-VP ngày 30/12/2024,NaN
1625,762.0,UBND thành phố Hà Giang,UBND thành phố,Phạm vi tiếp nhận thủ tục hành chính tại Bộ ph...,Bộ phận Tiếp nhận và Trả kết quả (Bộ phận Một ...,NaN,3370/UBND-VP ngày 30/12/2024,NaN


In [ ]:
len(df["Câu hỏi"])

1627

In [ ]:
# Giữ lại câu trả lời dài nhất cho mỗi câu hỏi
df = df.loc[df.groupby('Câu hỏi')['Câu trả lời'].apply(lambda x: x.str.len().idxmax())].reset_index(drop=True)
df

,STT,Tên cơ quan,Tên đơn vị,Câu hỏi,Câu trả lời,Tên TTHC,Số hiệu văn bản,Ghi Chú
0,705.0,Sở tư pháp,NaN,"Chào anh/chị, tôi có ứng tuyển tại một công t...",Việc đề nghị cấp phiếu LLTP số 1 hay số 2 là d...,Cấp Phiếu lý lịch tư pháp cho công dân Việt Na...,39/STP-VP ngày 10/01/2024,NaN
1,499.0,UBND huyện Mèo Vạc,UBND cấp xã,"1, Cho tôi hỏi Thủ tục tặng Giấy khen của Chủ ...","Bước 1. Cán bộ làm công tác thi đua, khen thưở...",Thủ tục tặng Giấy khen của Chủ tịch UBND cấp x...,29/BC-UBND ngày 11/01/2024,NaN
2,501.0,UBND huyện Mèo Vạc,UBND cấp xã,"1, Cho tôi hỏi Thủ tục tặng Giấy khen của Chủ ...","Bước 1. Cán bộ làm công tác thi đua, khen thưở...",Thủ tục tặng Giấy khen của Chủ tịch UBND cấp x...,29/BC-UBND ngày 11/01/2024,NaN
3,497.0,UBND huyện Mèo Vạc,UBND cấp xã,1. Cho tôi hỏi Thủ tục giải quyết tố cáo tại c...,Bước 1: Thụ lý tố cáo\nTrước khi thụ lý tố cáo...,Thủ tục giải quyết tố cáo tại cấp xã,29/BC-UBND ngày 11/01/2024,NaN
4,500.0,UBND huyện Mèo Vạc,UBND cấp xã,1. Cho tôi hỏi Thủ tục tặng Giấy khen của Chủ ...,"Bước 1. Cán bộ làm công tác thi đua, khen thưở...",Thủ tục tặng Giấy khen của Chủ tịch UBND cấp x...,29/BC-UBND ngày 11/01/2024,NaN
...,...,...,...,...,...,...,...,...
1419,239.0,Sở Giáo dục và Đào tạo,NaN,"Đối với học sinh đã được xét duyệt, trong các ...","Người học thuộc diện miễn, giảm học phí và hỗ ...","Đề nghị miễn giảm học phí, hỗ trợ chi phí học ...",37/SGDĐT-VP ngày 09/01/2024,NaN
1420,436.0,UBND huyện Mèo Vạc,Phòng Văn hóa và Thông tin,Đổi giấy chứng nhận đăng ký hoạt động của cơ s...,Cơ sở hỗ trợ nạn nhân bạo lực gia đình do các ...,Đổi Giấy chứng nhận đăng ký hoạt động của cơ s...,29/BC-UBND ngày 11/01/2024,NaN
1421,437.0,UBND huyện Mèo Vạc,Phòng Văn hóa và Thông tin,Đổi giấy chứng nhận đăng ký hoạt động của cơ s...,"Cơ sở tư vấn về phòng, chống bạo lực gia đình ...",Đổi Giấy chứng nhận đăng ký hoạt động của cơ s...,29/BC-UBND ngày 11/01/2024,NaN
1422,573.0,Sở Nội vụ,NaN,Đổi tên hội cần hoàn thiện thành phần hồ sơ nào?,Hồ sơ đổi tên hội theo quy định tại Khoản 2 Đi...,2001678,102/SNV-VP ngày 30/01/2024,NaN


In [ ]:
from llama_index.core import Document

documents_dachuanhoa = [
    Document(
        text= ', '.join([
            row["Câu hỏi"],
    ]),
        metadata={
            'code': row['STT'],
            'agencyName': row['Tên cơ quan'],
            'agencyLevel': row['Tên đơn vị'],
            'answer': row['Câu trả lời'],
            'name': row['Tên TTHC'],
            'documentNumber': row['Ghi Chú'],
        }
    )
    for _, row in df.iterrows() 
]

In [ ]:
len(documents_dachuanhoa)

1424

### Vector Stores

#### Creating a Weaviate Client

In [ ]:
# %pip install llama-index-vector-stores-weaviate

In [ ]:
# pip install -U weaviate-client

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
import weaviate

In [73]:
# import weaviate
# import os
  
# Connect to a WCS instance
# client = weaviate.connect_to_wcs(
#     cluster_url="cluster_url",
#     auth_credentials=weaviate.auth.AuthApiKey("API_KEY"))


In [ ]:
# local
client = weaviate.connect_to_local(
)

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"


In [9]:
print(client.is_ready())

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/ready "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/.well-known/ready "HTTP/1.1 200 OK"
True


### Indexing

In [13]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from IPython.display import Markdown, display

In [14]:
from llama_index.core import StorageContext

In [77]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

embed_model_weaviate = HuggingFaceEmbedding(
    model_name="maiduchuy321/vietnamese-bi-encoder-fine-tuning-for-law-chatbot",device="cuda",embed_batch_size=10
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: I:/Hoc/Public Administration/vietnamese-bi-encoder-fine-tuning-for-law-chatbot_epoch15
Load pretrained SentenceTransformer: I:/Hoc/Public Administration/vietnamese-bi-encoder-fine-tuning-for-law-chatbot_epoch15
Load pretrained SentenceTransformer: I:/Hoc/Public Administration/vietnamese-bi-encoder-fine-tuning-for-law-chatbot_epoch15
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


In [78]:
client.collections.delete("HCC_QuestionAndAnswer")  # Replace with your collection name

vector_store_HCC = WeaviateVectorStore(
    weaviate_client=client, index_name="HCC_QuestionAndAnswer", tenant="HCC_QuestionAndAnswer"
)
storage_context_HCC = StorageContext.from_defaults(vector_store=vector_store_HCC)

index_HCC = VectorStoreIndex(
    documents_dachuanhoa, embed_model=embed_model_weaviate, storage_context=storage_context_HCC
)

INFO:httpx:HTTP Request: DELETE http://localhost:8080/v1/schema/HCC_QuestionAndAnswer "HTTP/1.1 200 OK"
HTTP Request: DELETE http://localhost:8080/v1/schema/HCC_QuestionAndAnswer "HTTP/1.1 200 OK"
HTTP Request: DELETE http://localhost:8080/v1/schema/HCC_QuestionAndAnswer "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/schema/HCC_QuestionAndAnswer "HTTP/1.1 404 Not Found"
HTTP Request: GET http://localhost:8080/v1/schema/HCC_QuestionAndAnswer "HTTP/1.1 404 Not Found"
HTTP Request: GET http://localhost:8080/v1/schema/HCC_QuestionAndAnswer "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:8080/v1/schema "HTTP/1.1 200 OK"


Batches: 100%|██████████| 1/1 [00:00<00:00, 30.71it/s]


INFO:httpx:HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"


### Loading the index

In [68]:
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from IPython.display import Markdown, display

vector_store = WeaviateVectorStore(
    weaviate_client=client, index_name="HCC_QuestionAndAnswer"
)

loaded_index = VectorStoreIndex.from_vector_store(vector_store)
retriever_hybrid = loaded_index.as_retriever(
    vector_store_query_mode="hybrid",
    similarity_top_k=3,
    alpha=0.75 #very similar to vector search
    )

HTTP Request: GET http://localhost:8080/v1/schema/HCC_QuestionAndAnswer "HTTP/1.1 200 OK"


#### Query index

In [86]:
query = "Đăng ký kết hôn bao nhiêu ngày sẽ có kết quả?"

In [92]:
retriever = loaded_index.as_retriever(
    vector_store_query_mode="hybrid",
    similarity_top_k=3,
    alpha=0 #very similar to vector search
    )

In [93]:
nodes = retriever.retrieve(query)

Batches: 100%|██████████| 1/1 [00:00<00:00, 15.11it/s]

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/schema/HCC_QuestionAndAnswer "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/schema/HCC_QuestionAndAnswer "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/schema/HCC_QuestionAndAnswer "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/schema/HCC_QuestionAndAnswer "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/schema/HCC_QuestionAndAnswer "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/schema/HCC_QuestionAndAnswer "HTTP/1.1 200 OK"


In [94]:
texts = [item.text for item in nodes]
for index, _ in enumerate(texts, start=1):
    print(index, texts[index - 1])

1 Đăng ký kết hôn bao nhiêu ngày sẽ có kết quả?
2 Thời hạn đăng ký kết hôn có yếu tố nước ngoài hiện nay bao nhiêu ngày?
3 Thời hạn giải quyết đăng ký kết hôn có yếu tố nước ngoài bao nhiêu lâu ?


In [95]:
nodes[0].score

0.0

In [96]:
from llama_index.core.response.notebook_utils import display_source_node

for node in nodes:
    display_source_node(node, source_length=1024)

**Node ID:** 156e0c63-5d6d-42dd-acfa-42c1a65f0448<br>**Similarity:** 0.0<br>**Text:** Đăng ký kết hôn bao nhiêu ngày sẽ có kết quả?<br>

**Node ID:** 9aa54b0e-1803-4065-ac1d-91c0a2c4715a<br>**Similarity:** 0.45405739545822144<br>**Text:** Thời hạn đăng ký kết hôn có yếu tố nước ngoài hiện nay bao nhiêu ngày?<br>

**Node ID:** ec9b0dd6-d8de-4ade-838a-fdcc5deaea32<br>**Similarity:** 0.4826204776763916<br>**Text:** Thời hạn giải quyết đăng ký kết hôn có yếu tố nước ngoài bao nhiêu lâu ?<br>

In [52]:
print(nodes[0].text)
print(nodes[0].metadata['answer'])

Thời hạn giải quyết thủ tục đăng ký công bố hợp chuẩn dựa trên kết quả chứng nhận hợp chuẩn của tổ chức
 chứng nhận?
Theo quy định của pháp luật: 05 ngày làm việc kể từ ngày nhận đủ hồ sơ hợp lệ
Theo thực tế tại địa phương: 04 ngày làm việc kể từ ngày nhận đủ hồ sơ hợp lệ
